In [1]:
import pandas as pd
from datasets import load_dataset

# Load CSV as a Dataset
dataset = load_dataset("csv", data_files="tmp.csv", split="train", encoding="utf-8", quotechar='"')

# Helper functions
def align_teencode(ori, nor):
    teencode_words = []
    normalize_words = []
    for w1, w2 in zip(ori.split(), nor.split()):
        if w1 != w2:
            teencode_words.append(w1)
            normalize_words.append(w2)
    return teencode_words, normalize_words

def detect_teencode(ori, nor):
    return [w1 for w1, w2 in zip(ori.split(), nor.split()) if w1 != w2]

def replace_teencode(ori, nor):
    _, normalize_words = align_teencode(ori, nor)
    return {
        "Instruction": f"Cho các từ {normalize_words} và câu sau có chứa teencode tương ứng, hãy thay thế từ teencode bằng từ đã cho:",
        "Input": ori,
        "Answer": nor,
        "Task": "replacement"
    }

def generate_instruction_rows(ori, nor):
    rows = []

    # Template 1
    rows.append({
        "Instruction": "Hãy chuẩn hóa câu sau thành câu bình thường:",
        "Input": ori,
        "Answer": nor,
        "Task": "normalization"
    })

    # Template 2
    rows.append({
        "Instruction": "Cho cặp câu trong đó câu 1 là câu chứa các từ teencode và câu 2 là câu đã chuẩn hóa sai các từ teencode ở câu 1, hãy chuẩn hóa lại các từ bị chuẩn hóa sai",
        "Input": f"Câu 1: {ori}, Câu 2: {ori}",
        "Answer": f"Các từ bị chuẩn hóa sai: {detect_teencode(ori, nor)}, sửa lại: {nor}",
        "Task": "detection"
    })

    # Template 3
    rows.append({
        "Instruction": "Cho biết các từ nào là teencode trong câu sau:",
        "Input": ori,
        "Answer": f"Các từ teencode: {detect_teencode(ori, nor)}",
        "Task": "detection"
    })

    # Template 4
    rows.append(replace_teencode(ori, nor))

    return rows

# Generate all rows
all_rows = []
for ori, nor in zip(dataset['original'], dataset['normalized']):
    if not ori or not nor:
        continue
    all_rows.extend(generate_instruction_rows(ori, nor))

# Convert to DataFrame
df = pd.DataFrame(all_rows)

# Save to CSV
df.to_csv("instruction_tmp4.csv", index=False, encoding="utf-8")

Generating train split: 0 examples [00:00, ? examples/s]